In [1]:
from gams import *
import pandas as pd
from dreamtools.gams_pandas import *
from dreamtools.gamY import Precompiler
from abatement_functions import nesting_tree,a_model
import DataBase
import DB2Gams

# **Final goods sector**

This runs the final goods sector alone, to provide some understanding of relevant classes in the abatement model.

## **1: Define nesting structure and read in data via nesting_trees**

Define nesting structure of inputs by mapping: Keys are aggregates, values are inputs in that aggregate. 

In [2]:
FG_tree = nesting_tree(
    name='FG',
    tree = {
        'KE': ['K','E'],
        'E' : ['E1','E2']},
    **{'setname': 'n',
       'alias': 'nn',
       'alias2' : 'nnn',
       'inpname': 'in_FG',
       'aggname': 'agg_FG',
       'mapname': 'in_FG2agg_FG',
       'outname': 'out_FG',
       'sector':  'sec_FG'}
)

Create sets, subsets, and mappings from tree:

In [3]:
FG_tree.run_all()

Read in data from excel:

In [4]:
read_type = {'1dvars': ['sigma'], # list sheets with with data of variables defined over 1 set
             'vars_panel': {'mu': 2}} # dictionary with variables defined over n sets written in panel-data form. Key refers to sheet name, value refers to number of sets.
FG_tree.database.read_from_excel('Make_data//FinalGoods.xlsx',read_type)

Add other variables needed for the model to run (or read them in as done above):

## **2: Initialize abatement_model with nesting_tree**

In [5]:
model = a_model(FG_tree)
model.define_groups()
model.define_blocks()

With groups and blocks defined run the 'default' version, where files are created and written to a repo:

In [6]:
repo = os.getcwd()+'\\gms\\FG'
model.run_default(repo)

The default run writes the files, stores them in a repository of the user's choosing, and updates the model settings for later run.

### **3: Finally, use the model to initialize gams model**

Initialize model with database, and workfolder:

In [7]:
gams_model = DB2Gams.gams_model(databases = {model.model.name: model.database}, 
                                work_folder = os.getcwd()+'\\workfolder')

In [8]:
gams_model.run(model.model,'options')

The database:

In [11]:
gams_model.out_db.types

{'n': 'set',
 'alias_map2': 'set',
 'alias_set': 'set',
 'agg_FG': 'subset',
 'alias_': 'mapping',
 'out_FG': 'subset',
 'in_FG': 'subset',
 'in_FG2agg_FG': 'mapping',
 'sec_FG': 'subset',
 'sigma': 'variable',
 'mu': 'variable',
 'p': 'variable',
 'q': 'variable',
 'load_p': 'parameter',
 'load_q': 'parameter',
 'load_sigma': 'parameter',
 'load_mu': 'parameter',
 'E_FG_q': None,
 'E_FG_p': None}